In [1]:
import tensorflow as tf
import numpy as np
from numpy import linalg as LA
import numpy as np

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
x = tf.placeholder(tf.float32, shape=[784, None])
y_true = tf.placeholder(tf.float32, shape=[10, None])
W = tf.Variable(tf.zeros([10,784]))
b = tf.Variable(tf.zeros([10,1]))
y_pred = tf.matmul(W, x) + b
y_pred.get_shape()

TensorShape([Dimension(10), Dimension(None)])

In [4]:
# loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))
# optimzr = tf.train.AdamOptimizer(1e-4).minimize(loss)

# correct_prediction = tf.equal(tf.argmax(y_pred, 0), tf.argmax(y_true, 0))
# accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [5]:
def tf_frobenius_norm(M):
    return tf.reduce_sum(M ** 2) ** 0.5

# to implement nuclear norm
def tf_nuclear_norm(M):
    return 1

def Cgd_Fn(grad, wt):
    return ((1 - alpha ) / alpha) * (wt - lam1 * grad / tf_frobenius_norm(grad))

def Cgd_Nn(grad, wt):
    return ((1 - alpha ) / alpha) * (wt - lam2 * tf_nuclear_norm(grad))


In [20]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))

# hyper-parameters
alpha = tf.placeholder_with_default(tf.constant(0.5), tf.constant(0.5).shape)
lam1 = tf.placeholder_with_default(tf.constant(100.0), tf.constant(100.0).shape)
lam2 = tf.placeholder_with_default(tf.constant(100.0), tf.constant(100.0).shape)

# Gradient Descent optimizer
opt = tf.train.GradientDescentOptimizer(learning_rate = alpha)

# Compute the gradients for a list of variables.
grads_and_vars = opt.compute_gradients(loss)

# Normal GD update
gv_sgd = [(gv[0], gv[1]) for gv in grads_and_vars]
optimizer_gv_sgd = opt.apply_gradients(gv_sgd)
# qa = grads_and_vars[0][0]
# qb = tf_frobenius_norm(grads_and_vars[0][0])
# qc = grads_and_vars[0][0] / tf_frobenius_norm(grads_and_vars[0][0])
# qd = Cgd_Fn(grads_and_vars[0][0], grads_and_vars[0][1])
# qe = grads_and_vars[0][1]
# qf = ((1 - alpha ) / alpha)*(grads_and_vars[0][1] - lam1 * grads_and_vars[0][0] / tf_frobenius_norm(grads_and_vars[0][0]))
# qg = qe - alpha * qa

# CGD with FN
gv_cgd_fn = [(Cgd_Fn(gv[0], gv[1]), gv[1]) for gv in grads_and_vars]
optimizer_gv_cgd_fn = opt.apply_gradients(gv_cgd_fn)
# qa = grads_and_vars[0][0]
# qb = tf_frobenius_norm(grads_and_vars[0][0])
# qc = grads_and_vars[0][0] / tf_frobenius_norm(grads_and_vars[0][0])
# qd = Cgd_Fn(grads_and_vars[0][0], grads_and_vars[0][1])
# qe = grads_and_vars[0][1]
# qf = ((1 - alpha ) / alpha)*(grads_and_vars[0][1] - lam1 * grads_and_vars[0][0] / tf_frobenius_norm(grads_and_vars[0][0]))
# qg = qe - alpha * qf

# CGD with NN
gv_cgd_nn = [(Cgd_Nn(gv[0], gv[1]), gv[1]) for gv in grads_and_vars]
optimizer_gv_cgd_Nn = opt.apply_gradients(gv_cgd_nn)

correct_prediction = tf.equal(tf.argmax(y_pred, 0), tf.argmax(y_true, 0))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [25]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1000):
        batch = mnist.train.next_batch(100)
        feed_dict = {x: np.transpose(batch[0]), y_true: np.transpose(batch[1])}
        train_accuracy, loss_, gv_, W_ = sess.run([accuracy, loss, gv_sgd, W], feed_dict)
#         qa_,qb_,qc_,qd_,qe_,qf_, qg_ = sess.run([qa,qb,qc,qd,qe,qf,qg], feed_dict)
            
        if i % 100 == 0:
            print('train_accuracy', train_accuracy, loss_)
#             print('frob_nrom: qg =', LA.norm(qg_), 'qe =', LA.norm(qe_), 'qf =', LA.norm(qf_))
#             print('gradient', LA.norm(qa_))
        sess.run(optimizer_gv_sgd, feed_dict)
    
    feed_dict={x: np.transpose(mnist.test.images), y_true: np.transpose(mnist.test.labels)}
    test_accuracy = sess.run(accuracy, feed_dict)
    print('test_accuracy', test_accuracy)

train_accuracy 0.12 46.0517
frob_nrom: qg = 50.0 qe = 0.0 qf = 100.0
gradient 20.9454
train_accuracy 0.3 115.304
frob_nrom: qg = 45.5473 qe = 10.4727 qf = 109.184
gradient 21.4472
train_accuracy 0.29 196.457
frob_nrom: qg = 41.7969 qe = 20.4676 qf = 117.686
gradient 20.5409
train_accuracy 0.22 279.868
frob_nrom: qg = 38.6852 qe = 29.6106 qf = 125.568
gradient 21.1563
train_accuracy 0.29 297.393
frob_nrom: qg = 36.4559 qe = 38.8075 qf = 133.026
gradient 21.4779
train_accuracy 0.57 409.336
frob_nrom: qg = 30.1551 qe = 47.8129 qf = 144.689
gradient 19.6404
train_accuracy 0.57 439.668
frob_nrom: qg = 28.7238 qe = 56.8492 qf = 152.195
gradient 20.1504
train_accuracy 0.54 574.349
frob_nrom: qg = 25.7552 qe = 65.8329 qf = 161.291
gradient 20.635
train_accuracy 0.57 614.848
frob_nrom: qg = 20.6244 qe = 75.137 qf = 172.017
gradient 20.0766
train_accuracy 0.5 711.686
frob_nrom: qg = 19.5511 qe = 84.5346 qf = 181.006
gradient 19.4743
train_accuracy 0.52 837.488
frob_nrom: qg = 21.9007 qe = 93.603

In [8]:
print(type(grads_and_vars[0][0]))

<class 'tensorflow.python.framework.ops.Tensor'>


In [9]:
a = tf.constant([[1,2],[3,4],[5,6]])
b = a**2
tf.Session().run(b)

array([[ 1,  4],
       [ 9, 16],
       [25, 36]], dtype=int32)